In [ ]:
pip install google-cloud-texttospeech

In [3]:
import os
from google.cloud import texttospeech
from openai import OpenAI
from tempfile import NamedTemporaryFile

# Function to prompt the user for themes
def get_themes():
    themes = input("Enter themes to create a story around (separated by commas): ")
    return themes.split(',')

# Point to the local OpenAI server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

try:
    # Get themes from the user
    themes = get_themes()

    # Construct prompt
    prompt = "Create an engaging, education, 500+ words story about"
    for theme in themes:
        prompt += f" {theme.strip()},"

    # Add system message for rhyming
    prompt += "\nAlways answer in rhymes."

    # Generate completion
    completion = client.chat.completions.create(
        model="bartowski/Young-Children-Storyteller-Mistral-7B-GGUF",
        messages=[
            {"role": "system", "content": "Always answer with a complete, engaging, educational young children story."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
    )

    # Print the generated story
    generated_story = completion.choices[0].message.content
    print("Generated Story:")
    print(generated_story)

    # Use Google TTS API to convert the story to speech
    google_client = texttospeech.TextToSpeechClient()
    synthesis_input = texttospeech.SynthesisInput(text=generated_story)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",  # Example language code
        ssml_gender=texttospeech.SsmlVoiceGender.FEMALE
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = google_client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )

    # Save the narrated story as an MP3 file on the desktop
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    mp3_file_path = os.path.join(desktop_path, "narrated_story.mp3")
    with open(mp3_file_path, "wb") as mp3_file:
        mp3_file.write(response.audio_content)

    print("Narration MP3 file saved at:", mp3_file_path)

except Exception as e:
    print("An error occurred:", e)

Enter themes to create a story around (separated by commas): ocean, dolphin, ice
Generated Story:
  Once upon a time, in the vast and wondrous **OCEAN**, lived many magical creatures. But none were as playful and intelligent as our dear friend **DOLPHIN**. With their shiny black bodies and bright smiley faces, they could make anyone's day brighter!

One sunny morning, little Timmy sat by the shore watching the gentle waves roll in. Suddenly, he saw something silver glittering amidst the foam. It was Dory, a friendly dolphin who loved making new friends! She swam up to Timmy and said, "Hello there, I see you admiring my home."

"Your what?" asked Timmy curiously.

"Why, the big blue place we live in," replied Dory cheerfully. "It's called the ocean!" And so she took him on a journey beneath the waves, showing all sorts of marvelous things - colorful coral reefs teeming with fish, sea turtles gracefully diving deep into dark waters, and even a giant octopus changing colors like a chamele

In [2]:
pip install gradio

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for websockets<12.0,>=10.0 from https://files.pythonhosted.org/packages/e1/7c/0ad6e7ef0a054d73092f616d20d3d9bd3e1b837554cb20a52d8dd9f5b049/websockets-11.0.3-cp311-cp311-win_amd64.whl.metadata
  Using cached websockets-11.0.3-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
Using cached websockets-11.0.3-cp311-cp311-win_amd64.whl (124 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yt-dlp 2023.12.30 requires websockets>=12.0, but you have websockets 11.0.3 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import gradio as gr
from google.cloud import texttospeech
from openai import OpenAI
from tempfile import NamedTemporaryFile

# Point to the local OpenAI server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def generate_story(themes):
    # Construct prompt
    prompt = "Create an engaging, education, 500+ words story about"
    for theme in themes.split(','):
        prompt += f" {theme.strip()},"

    # Add system message for rhyming
    prompt += "\nAlways answer in rhymes."

    # Generate completion
    completion = client.chat.completions.create(
        model="bartowski/Young-Children-Storyteller-Mistral-7B-GGUF",
        messages=[
            {"role": "system", "content": "Always answer with a complete, engaging, educational young children story."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
    )

    # Extract the generated story
    generated_story = completion.choices[0].message.content

    # Use Google TTS API to convert the story to speech
    google_client = texttospeech.TextToSpeechClient()
    synthesis_input = texttospeech.SynthesisInput(text=generated_story)
    voice = texttospeech.VoiceSelectionParams(
        language_code="en-US",
        ssml_gender=texttospeech.SsmlVoiceGender.FEMALE
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3
    )
    response = google_client.synthesize_speech(
        input=synthesis_input, voice=voice, audio_config=audio_config
    )

    # Save the narrated story as a temporary MP3 file
    temp_file = NamedTemporaryFile(delete=False, suffix='.mp3')
    temp_file.write(response.audio_content)
    temp_file.close()

    return generated_story, temp_file.name

def cleanup(file_path):
    os.remove(file_path)

# Define the Gradio interface
iface = gr.Interface(
    fn=generate_story,
    inputs=[gr.Textbox(label="Enter themes to create a story around (separated by commas)"), gr.Checkbox(label="Cleanup MP3 after playback", value=True)],
    outputs=[
        gr.Textbox(label="Generated Story"),
        gr.Audio(label="Narrated Story MP3", type="filepath", optional=True)
    ],
    examples=[["adventure, magic, friendship"]],
    cleanup=cleanup
)

# Launch the interface
iface.launch()


AttributeError: partially initialized module 'fsspec' has no attribute 'utils' (most likely due to a circular import)